### Load Functions

In [1]:
from parameter import *
from parameter import split_data,model_build_train, model_predict, trace_seq_path, train_batch, anomaly_match

### Load Data and Path

In [2]:
df = pd.read_csv(PurePosixPath('Drain_result/RtpDump.log_structured.csv'))
model_file = PurePosixPath("para_model.h5")
scaler_file =  ''
label_file =  ''
trace_df = pd.read_csv(PurePosixPath('EPA_trace_df.csv'))
trace_df

,Unnamed: 0,log message,log key,parameter value vector,seq_path,path_pred
0,0,3 18 16:34:25 Cluster observer forked - pid = ...,"Cluster observer forked - pid = <*>""","[0, '1386']","[17.0, 24.0, 36.0, 30.0, 44.0, 31]",0
1,1,3 18 16:34:25 Initialization successful. Start...,Initialization successful. Starting now the su...,[0],"[24.0, 36.0, 30.0, 44.0, 31.0, 30]",0
2,2,3 18 16:34:25 SNM Cluster observer starting (P...,"SNM Cluster observer starting (PID <*>)...""","[0, '1386']","[36.0, 30.0, 44.0, 31.0, 30.0, 30]",0
3,3,3 18 16:34:25 Process RtpTrcMgr01 is fully fun...,"Process <*> is fully functional.""","[0, 'RtpTrcMgr01']","[30.0, 44.0, 31.0, 30.0, 30.0, 43]",0
4,4,3 18 16:34:25 We have 1 configured nodes in th...,"We have <*> configured nodes in the cluster""","[0, '1']","[44.0, 31.0, 30.0, 30.0, 43.0, 30]",0
...,...,...,...,...,...,...
95,95,3 18 16:34:28 Alias RtpCtx01_00 changed its st...,Alias <*> changed its state; new primary: <*> ...,"[0, 'RtpCtx01_00', 'RtpCtxMaster01_00,']","[3.0, 3.0, 42.0, 30.0, 25.0, 25]",0
96,96,3 18 16:34:28 Alias RtpCtx01_00 changed its st...,Alias <*> changed its state; new primary: <*> ...,"[0, 'RtpCtx01_00', 'RtpCtxMaster01_00,']","[3.0, 42.0, 30.0, 25.0, 25.0, 25]",0
97,97,3 18 16:34:28 The process was successfully ini...,The process was successfully initialized and i...,[0],"[42.0, 30.0, 25.0, 25.0, 25.0, 25]",0
98,98,3 18 16:34:29 Process RtpRestServ is fully fun...,"Process <*> is fully functional.""","[0, 'RtpRestServ']","[30.0, 25.0, 25.0, 25.0, 25.0, 32]",0


### Transform to Numerical Data for each log key 

In [ ]:
import gen_matrics

fd_values = pd.read_csv('data/RtpDump.log_value_vector.csv')
# create the aim file where the key_num_para_dict.csv will be saved
para_dict_filename = 'temp/aim.csv'
key_num_para_dict = gen_matrics.log_vectors(fd_values, para_dict_filename)

i = 0
# create all the matrixes for all the eventIDs
for key in key_num_para_dict.keys():
    print("the key is:", key)
    gen_matrics.str_array(key_num_para_dict, key, 'temp/npy2/', str(i))
    i+=1

###  Select specific log key Parameters 

In [ ]:
# load testing parameter value vector
eventId = 16
# with categorical data inside
data = df[df['EventTemplate']=='A <*> <*> was <*>']['ParameterList']
    
# feature engineering for parameter value matrix
col_num = len(data[0])
    
new_data = []

In [ ]:
# feature engineering for parameter value matrix
data = np.load('temp/npy/E31.npy')
col_num = len(data[0])
new_data = []
data

### Feature Engineering

In [ ]:
# feature engineering for every single column
for col_ord in range(col_num):
        
    new_data.append([row[col_ord] for row in data])
    # replace the missing values
    new_data[col_ord] = miss_rep_col(new_data[col_ord])

    # create paths to save encoder model
    label_encoder_path = Path(label_file).joinpath(str(eventId), str(col_ord) + 'label.save')
        
    if not Path(label_encoder_path).parent.is_dir():
        Path(label_encoder_path).parent.mkdir(parents=True, exist_ok=True)
        
    # encode categorical labels
    if pd.Series(new_data[col_ord]).dtype == 'O':
        new_data[col_ord] = lab_enc(new_data[col_ord], label_encoder_path.as_posix())            
        
    # nomalize the column
    # new_data[col_ord] = 
    # stan_cols(new_data[col_ord], col_ord, eventId, scaler_file)
        
    # reshape 2D to 1D
    new_data[col_ord] = np.reshape(new_data[col_ord],new_data[col_ord].shape[0])

### Split into X[5], y[1]

In [ ]:
# shift the row to column   
new_data = np.array(new_data).T
n_steps = 5
X, y = split_data(new_data, n_steps)
X.shape, y.shape

### Reshape data and Build model

In [ ]:
%%time 
# reshape x to (samples, time steps, features)
train_X = np.array(X).reshape(-1, n_steps, len(data[0]))
# reshape y to (samples, features)
train_y = np.array(y).reshape(-1, len(data[0]))
train_X.shape, train_y.shape

In [ ]:
%time model = model_build_train(train_X, train_y, model_file)

### MSE Error Prediction

In [ ]:
# model = keras.models.load_model(model_file)
%time mse_error = model_predict(model, train_X[:50], train_y[:50])
print(mse_error)

### Find Anomaly (if out of confidence)

In [ ]:
# confidence = 0.99
# print(confidence_interval(confidence, mse_error))
fp_int = 0.97 
tp_int = 0.999
attempts = 10
threshold1, threshold3, seq_pre_dict = anomaly_match(mse_error, fp_int, tp_int, eventId)

In [ ]:
visual_mses(eventId, mse_error, threshold1, threshold3, fp_int, tp_int)

### Mark Anomalies

In [ ]:
lab_encoder_file = PurePosixPath("encoder.save")
print(f'Trace File Size: {trace_df.shape},  Predicted Seq. Size: {seq_pre_dict}')
trace_df = trace_seq_path(trace_df, seq_pre_dict, eventId, lab_encoder_file)
trace_df.to_csv(PurePosixPath('param_trace_df.csv'),index=False)

### Online Learning

In [ ]:
steps = 5
%time train_batch(model, model_file, train_X[:50], train_y[:50], steps, tp_int, attempts)